### dependencies

In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
from keras.utils import np_utils
from tensorflow.keras import optimizers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, Activation
import gc
import warnings
warnings.filterwarnings('ignore')

### creating the dataset

In [2]:
path = '../input/amazonsatelliteimages/train-jpg/train-jpg'

for img in os.listdir(path):
    img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
    plt.imshow(img_arr)
    # printing one image
    break

In [3]:
gc.collect()
IMG_SIZE = 50

train_data = []

def create_train_data():

    for img in os.listdir(path):
        
        try:
            img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
            new_img_arr = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))
            train_data.append(new_img_arr)
        except Exception as e:
            print(e)
        
create_train_data()

In [4]:
labels = pd.read_csv('../input/planet-understanding-the-amazon-from-space/train_v2.csv/train_v2.csv')

X = np.array(train_data)
y = pd.DataFrame(labels['tags'])

X.shape

### encoding output variable

In [5]:
encode = LabelEncoder().fit(y)
encoded_y = encode.transform(y)
y = np_utils.to_categorical(encoded_y)

pickle.dump(X, open('./X.pkl','wb'))
pickle.dump(y, open('./y.pkl','wb'))

y.shape

### creating the model

In [6]:
train_X = pickle.load(open('./X.pkl', 'rb'))
train_y = pickle.load(open('./y.pkl', 'rb'))

# normalizing features
train_X = train_X/255.0

n_epochs = 70
b_size = 1000

model = Sequential()
# ADDING LAYERS

# input layer
model.add(Conv2D(64, (3, 3), padding='same', input_shape=train_X.shape[1:]))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

# output neuron
model.add(Flatten())
model.add(Dense(4096,activation="relu"))
model.add(Dense(4096,activation="relu"))
model.add(Dense(4096,activation="relu"))

model.add(Dense(449))
model.add(Activation('softmax'))

opt = optimizers.Adam(lr=1e-4)
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
gc.collect()
model.summary()

### training the model

In [8]:
gc.collect()

In [9]:
train = model.fit(train_X, train_y, epochs=n_epochs, batch_size=b_size, validation_split=0.5)
train
plt.figure()
plt.plot(train.history['loss'], 'blue')
plt.plot(train.history['val_loss'], 'red')
plt.legend(['Training Loss', 'Validation Loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss')

### creating test set

In [10]:
path = '../input/amazonsatelliteimages/test-jpg/test-jpg'
test_set = []

def create_test_data():

    for img in os.listdir(path):
        
        try:
            img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
            new_img_arr = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))
            test_set.append(new_img_arr)
        except Exception as e:
            print(e)
        
create_test_data()

test_x = np.array(train_data)


### making predictions

In [19]:
y_pred = model.predict(test_x)
y_pred = pd.DataFrame(y_pred)
y_pred.to_csv('./submit.csv')

### calculating fbeta score

In [20]:
test_y = train_y

In [26]:
# from sklearn.metrics import fbeta_score
# fbeta_score(test_y, y_pred, average='weighted', beta=0.5)

def fbeta(test_y, y_pred, beta=2, threshold=0.2, epsilon=1e-7):
    # threshold is set to 0.2 to maximize recall since f2 score is recall biased
    # epsilon is set to 1e-7 to avoide Nan values due to zero division
    
    beta_squared = float(beta)**2
    
    test_y = tf.cast(test_y, tf.float32) # casts test_y as a float
    # convert y_pred to bool, then to float
    y_pred = tf.cast(tf.greater(tf.cast(y_pred, tf.float32), tf.constant(threshold)), tf.float32) 
    
    tp = tf.reduce_sum(tf.cast(tf.equal((2.0*test_y + y_pred), tf.constant(3.0)), tf.float32), axis=1) 
    fp = tf.reduce_sum(tf.cast(tf.equal((2.0*test_y + y_pred), tf.constant(1.0)), tf.float32), axis=1)
    fn = tf.reduce_sum(tf.cast(tf.equal((2.0*test_y + y_pred), tf.constant(2.0)), tf.float32), axis=1)

    precision = tp / (tp+fp)
    recall = tp / (tp+fn)
    fb = (beta_squared+1) * precision * recall / (precision*beta_squared + recall + epsilon)
    
    print(fb)
    return fb

fbeta(test_y, y_pred)